In [1]:
import os

import mala
from mala import printout

In [2]:
parameters = mala.Parameters()

# Currently, the splitting in training, validation and test set are
# done on a "by snapshot" basis.
parameters.data.data_splitting_type = "by_snapshot"
parameters.data.use_graph_data_set = True

# Specify the training parameters.
parameters.running.max_number_epochs = 100
parameters.running.ldos_grid_batch_size = 40
parameters.running.learning_rate = 0.00001
parameters.running.trainingtype = "Adam"

parameters.targets.ldos_gridsize = 201
parameters.verbosity = 1

In [3]:
data_handler = mala.DataHandlerGraph(parameters)

# for i in range(16):
for i in range(1):
    data_handler.add_raw_snapshot(
        f'/bigdata/casus/wdm/Bartek_H2/H128/snapshot{i}/H_snapshot{i}.pw.scf.in',
        f'/bigdata/casus/wdm/Bartek_H2/H128/ldos/H_snapshot{i}.out.npy',
        (90, 90, 60, 201), 'tr'
    )
# for i in range(16, 20):
for i in range(1, 2):
    data_handler.add_raw_snapshot(
        f'/bigdata/casus/wdm/Bartek_H2/H128/snapshot{i}/H_snapshot{i}.pw.scf.in',
        f'/bigdata/casus/wdm/Bartek_H2/H128/ldos/H_snapshot{i}.out.npy',
        (90, 90, 60, 201), 'va'
    )

data_handler.prepare_data(reparametrize_scaler=False)

No data rescaling will be performed.
No data rescaling will be performed.
Checking the snapshots and your inputs for consistency.
Consistency check successful.
Data scalers already initilized, loading data to RAM.
Build datasets.


100%|██████████| 12150/12150 [00:44<00:00, 273.81it/s]


Build dataset: Done.


In [4]:
parameters.network.nn_type = "se3_transformer"
parameters.network.layer_sizes = [
    data_handler.input_dimension,
    64,
    data_handler.output_dimension
]

# Setup network and trainer.
network = mala.Network(parameters)
# trainer = mala.Trainer(parameters, network, data_handler)

In [6]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=10)
trainer.fit(network, data_handler)

TypeError: forward() missing 1 required positional argument: 'grid_graph'